<a href="https://colab.research.google.com/github/Nourshaolin/Real-Time-Retail-Analytics-and-Product-Recommender/blob/main/RealTime_Retail_Analytics_and_Product_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real-Time Retail Analytics and Product Recommender
Team members:
* Nour El Moujahid
* Hiba Jebbar
* Ilyass Louzali
* Aya Boumelha

##Milestone 1:

Neural Network Workflow (PyTorch): Train deep learning models to predict
purchasing behavior and customer retention based on session logs and historical purchase
data.


In [1]:
x = 1